In [1]:
server_args = []
# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

from manipulation import running_as_notebook


# Imports
import numpy as np
import pydot
from ipywidgets import Dropdown, Layout
from IPython.display import display, HTML, SVG

from pydrake.all import (
    AddMultibodyPlantSceneGraph, ConnectMeshcatVisualizer, DiagramBuilder, 
    FindResourceOrThrow, GenerateHtml, InverseDynamicsController, 
    MultibodyPlant, Parser, Simulator)
from pydrake.multibody.jupyter_widgets import MakeJointSlidersThatPublishOnCallback
import pydrake
from pydrake import geometry
from pydrake.math import RigidTransform, RollPitchYaw, RotationMatrix 
from pydrake.solvers.mathematicalprogram import MathematicalProgram, Solve
from pydrake.systems.jupyter_widgets import PoseSliders, WidgetSystem
from ipywidgets import ToggleButton, ToggleButtons
from functools import partial
from pydrake.all import (
    JointIndex, PiecewisePolynomial, JacobianWrtVariable,
    eq, ge,  AutoDiffXd, autoDiffToGradientMatrix, SnoptSolver, IpoptSolver,
    initializeAutoDiffGivenGradientMatrix, autoDiffToValueMatrix, autoDiffToGradientMatrix,
    AddUnitQuaternionConstraintOnPlant, PositionConstraint, OrientationConstraint,
    MeshcatContactVisualizer
)

import pickle

In [2]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=1e-4)

In [3]:
#add table
ground = pydrake.multibody.parsing.Parser(plant, scene_graph).AddModelFromFile("model/ground.urdf", "ground")

#add box
cracker_box = pydrake.multibody.parsing.Parser(plant, scene_graph).AddModelFromFile(
                "model/003_cracker_box_plannar_test.urdf", "cracker_box")

X_WB = RigidTransform(RotationMatrix.MakeXRotation(np.pi/2), [0., 0., 0.104200 + 1e-07])#
planar_joint_frame = plant.AddFrame(pydrake.multibody.tree.FixedOffsetFrame("planar_joint_frame", plant.world_frame(), X_WB))
box_joint = plant.AddJoint(pydrake.multibody.tree.PlanarJoint("box_joint", planar_joint_frame, plant.GetFrameByName("base_link_cracker", cracker_box)))
box_joint.set_default_translation([0, 0])


In [4]:
def AddPointFinger(plant):
    mu = 1.0
    finger = plant.AddModelInstance("finger")
    false_body1 = plant.AddRigidBody("false_body1", finger, pydrake.multibody.tree.SpatialInertia(0, [0,0,0], pydrake.multibody.tree.UnitInertia(0,0,0)))
    finger_body = plant.AddRigidBody("body", finger, pydrake.multibody.tree.SpatialInertia(mass=1.0, p_PScm_E=np.array([0., 0., 0.]),
            G_SP_E=pydrake.multibody.tree.UnitInertia(1.0, 1.0, 1.0)))
    shape = pydrake.geometry.Sphere(0.01)
    if plant.geometry_source_is_registered():
        plant.RegisterCollisionGeometry(finger_body, RigidTransform(), shape, "body", pydrake.multibody.plant.CoulombFriction(mu, mu))
        plant.RegisterVisualGeometry(finger_body, RigidTransform(), shape, "body", [.9, .5, .5, 1.0])
    finger_x = plant.AddJoint(pydrake.multibody.tree.PrismaticJoint("finger_x", plant.world_frame(), plant.GetFrameByName("false_body1"), [1, 0, 0], -1.0, 1.0))
    plant.AddJointActuator("finger_x", finger_x)
    finger_x.set_default_translation(-0.1)
    finger_z = plant.AddJoint(pydrake.multibody.tree.PrismaticJoint("finger_z", plant.GetFrameByName("false_body1"), plant.GetFrameByName("body"), [0, 0, 1], 0., 1.0))
    finger_z.set_default_translation(0.05)
    plant.AddJointActuator("finger_z", finger_z)

    return finger
finger = AddPointFinger(plant)

In [5]:
plant.Finalize()
plant.set_name("plant")

In [6]:
cracker_box_body = plant.GetBodyByName("base_link_cracker", cracker_box)
cracker_box_id = plant.GetBodyFrameIdIfExists(cracker_box_body.index())


draw_frames = True
frames_to_draw = [cracker_box_id] if draw_frames else []
meshcat = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, frames_to_draw=frames_to_draw, axis_length=0.15,
                 axis_radius=0.001,)

# #visualizer the contact
# contact_visualizer = builder.AddSystem(MeshcatContactVisualizer(meshcat, 
#                                        force_threshold=0.0,
#                                        contact_force_scale=0.05, 
#                                        contact_force_radius=0.001,   
#                                        plant=plant))
# builder.Connect(plant.get_contact_results_output_port(),
#                 contact_visualizer.GetInputPort("contact_results"))


diagram = builder.Build()
meshcat.load()


Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6001...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7001/static/
Connected to meshcat-server.


In [7]:
tmpfolder = 'tmp/'
with open(tmpfolder +  'test_finger_box_sliding_friction_2d_posa/sol.pkl', 'rb' ) as file:
    h_ini, q_o_ini, v_o_ini, vdot_o_ini, q_r_ini, v_r_ini, vdot_r_ini, u_ini,lambd_z_ini, lambd_x_pos_ini, lambd_x_neg_ini, gama_ini = pickle.load( file )

In [8]:
t_sol = np.cumsum(np.concatenate(([0],h_ini)))
q_o_sol = PiecewisePolynomial.FirstOrderHold(t_sol, q_o_ini)
q_r_sol = PiecewisePolynomial.FirstOrderHold(t_sol, q_r_ini)

In [9]:
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)

# meshcat.load()
# diagram.Publish(context)


meshcat.start_recording()
t0 = t_sol[0]
T = t_sol[-1]
for t in np.hstack((np.arange(t0, T, meshcat.draw_period), T)):
    context.SetTime(t)
    plant.SetPositions(plant_context, cracker_box, q_o_sol.value(t))
    plant.SetPositions(plant_context, finger, q_r_sol.value(t))
    diagram.Publish(context)

meshcat.stop_recording()
meshcat.publish_recording()